In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [2]:
data = pd.read_csv('/content/judge-1377884607_tweet_product_company.csv', encoding='latin-1')

In [3]:
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [4]:
# Selecting only the 'tweet_text' and 'is_there_an_emotion_directed_at_a_brand_or_product' columns for sentiment analysis

data=data[['tweet_text','is_there_an_emotion_directed_at_a_brand_or_product']]

In [5]:
# Rename columns for simplicity

data.columns = ['tweet', 'sentiment']

In [6]:
data.shape

(9093, 2)

In [7]:
pd.set_option('display.max_colwidth',None)


In [8]:
data.head()

,tweet,sentiment
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",Negative emotion
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,Positive emotion
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,Negative emotion
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Positive emotion


In [9]:
data['sentiment'].value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: sentiment, dtype: int64

In [10]:
# data preprocessing

In [11]:
data['tweet'].isnull().sum()

1

In [12]:
data['tweet'].fillna('', inplace=True)

In [13]:
# feature extraction

In [14]:
x = data['tweet']
y = data['sentiment']

In [15]:
y

0                         Negative emotion
1                         Positive emotion
2                         Positive emotion
3                         Negative emotion
4                         Positive emotion
                       ...                
9088                      Positive emotion
9089    No emotion toward brand or product
9090    No emotion toward brand or product
9091    No emotion toward brand or product
9092    No emotion toward brand or product
Name: sentiment, Length: 9093, dtype: object

In [16]:
# encoding

In [17]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)

In [18]:
y

array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [19]:
# Tokenize the text data using Keras Tokenizer

In [20]:
from keras.preprocessing import text
tokenizer=text.Tokenizer()
tokenizer.fit_on_texts(list(data['tweet']))
tokenized_text=tokenizer.texts_to_sequences(data['tweet'])

In [21]:
len(tokenized_text[0])

24

In [22]:
len(tokenized_text[1])

22

In [23]:
# Pad the tokenized_text to make all text sequences the same length (100)

In [24]:
from keras.utils import pad_sequences
x=pad_sequences(tokenized_text,maxlen=100)

In [25]:
# train_test_split

In [26]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [27]:
# SimpleRNN

In [28]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding,SimpleRNN,Dropout

In [29]:
len(tokenizer.word_index)

10147

In [30]:
# Create a Sequential model
model = Sequential()

# Add an Embedding layer
model.add(Embedding(input_dim = len(tokenizer.word_index)+1, output_dim=128, input_length=100))

# Add a SimpleRNN layer with 32 units

model.add(SimpleRNN(32))
model.add(Dropout(0.5))

# Add a Dense layer with 50 units
#model.add(Dense(50,activation = 'relu'))
#model.add(Dropout(0.5))

# Add the final Dense layer with 4 units (for 4 classes)
model.add(Dense(4, activation='softmax'))

In [31]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          1298944   
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                5152      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 4)                 132       
                                                                 
Total params: 1304228 (4.98 MB)
Trainable params: 1304228 (4.98 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
model.fit(x_train,y_train,epochs=10,validation_split=0.1)

Epoch 1/10
205/205 [==============================] - 12s 52ms/step - loss: 0.9810 - accuracy: 0.5458 - val_loss: 0.8546 - val_accuracy: 0.6277
Epoch 2/10
205/205 [==============================] - 10s 49ms/step - loss: 0.7219 - accuracy: 0.7236 - val_loss: 0.8763 - val_accuracy: 0.6319
Epoch 3/10
205/205 [==============================] - 9s 43ms/step - loss: 0.4494 - accuracy: 0.8524 - val_loss: 0.9722 - val_accuracy: 0.6277
Epoch 4/10
205/205 [==============================] - 10s 48ms/step - loss: 0.3283 - accuracy: 0.8935 - val_loss: 1.0702 - val_accuracy: 0.6085
Epoch 5/10
205/205 [==============================] - 10s 50ms/step - loss: 0.2582 - accuracy: 0.9148 - val_loss: 1.1021 - val_accuracy: 0.6154
Epoch 6/10
205/205 [==============================] - 10s 47ms/step - loss: 0.2171 - accuracy: 0.9247 - val_loss: 1.1606 - val_accuracy: 0.6264
Epoch 7/10
205/205 [==============================] - 10s 49ms/step - loss: 0.1943 - accuracy: 0.9314 - val_loss: 1.1955 - val_accuracy: 

In [34]:
y_pred=model.predict(x_test)

57/57 [==============================] - 1s 8ms/step


In [35]:
accuracy = model.evaluate(x_test, y_test)[1]
print(f'Test Accuracy: {accuracy * 100:.2f}%')

57/57 [==============================] - 0s 8ms/step - loss: 1.4061 - accuracy: 0.5871
Test Accuracy: 58.71%


In [36]:
#LSTM

In [37]:
model = Sequential()
model.add(Embedding(input_dim = len(tokenizer.word_index)+1, output_dim=128, input_length=100))
model.add(LSTM(32))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10,validation_split=0.1)

Epoch 1/10
205/205 [==============================] - 27s 118ms/step - loss: 0.9163 - accuracy: 0.6048 - val_loss: 0.8232 - val_accuracy: 0.6387
Epoch 2/10
205/205 [==============================] - 14s 71ms/step - loss: 0.6915 - accuracy: 0.7195 - val_loss: 0.7903 - val_accuracy: 0.6703
Epoch 3/10
205/205 [==============================] - 14s 71ms/step - loss: 0.4789 - accuracy: 0.8219 - val_loss: 0.8859 - val_accuracy: 0.6264
Epoch 4/10
205/205 [==============================] - 14s 70ms/step - loss: 0.3606 - accuracy: 0.8659 - val_loss: 0.9178 - val_accuracy: 0.6758
Epoch 5/10
205/205 [==============================] - 14s 70ms/step - loss: 0.2886 - accuracy: 0.8886 - val_loss: 1.0548 - val_accuracy: 0.6786
Epoch 6/10
205/205 [==============================] - 14s 70ms/step - loss: 0.2463 - accuracy: 0.9024 - val_loss: 1.0620 - val_accuracy: 0.6552
Epoch 7/10
205/205 [==============================] - 14s 71ms/step - loss: 0.2062 - accuracy: 0.9169 - val_loss: 1.1280 - val_accuracy

In [38]:
y_pred=model.predict(x_test)

57/57 [==============================] - 1s 13ms/step


In [39]:
accuracy = model.evaluate(x_test, y_test)[1]
print(f'Test Accuracy: {accuracy * 100:.2f}%')

57/57 [==============================] - 1s 14ms/step - loss: 1.3743 - accuracy: 0.6449
Test Accuracy: 64.49%
